In [1]:
import os
os.environ['TF_DEVICE_MIN_SYS_MEMORY_IN_MB'] = '128' 

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
import os

import pandas as pd

import librosa
import librosa.display

import numpy as np

import IPython.display as ipd

import matplotlib.pyplot as plt

import random

from collections import Counter

from sklearn.model_selection import train_test_split

import torch
import torchaudio

from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import DatasetDict, IterableDatasetDict
from datasets import Dataset as DS
from datasets import IterableDataset as IDS

from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainerCallback,
    TrainingArguments,
    TrainerState,
    TrainerControl,
    EarlyStoppingCallback,
    pipeline
)

from torchmetrics.text import WordErrorRate, CharErrorRate

In [3]:
BASE_DIR = "/media/ahanaf/media-1/Datasets/ben10/ben10/"
train_data_dir = f"{BASE_DIR}/16_kHz_train_audio/"
test_data_dir = f"{BASE_DIR}/16_kHz_valid_audio/"
data_path = f"{BASE_DIR}/train.csv"

In [4]:
split2path = {
    "train": train_data_dir,
    "test": test_data_dir,
}

In [5]:
data = pd.read_csv(data_path)
data.sample(10)

,file_name,transcripts
13339,train_tangail (787).wav,<> খবর দেহি দিন-রাইত চব্বিশ ঘণ্টা। দ্যাশের কোন...
9889,train_sylhet (282).wav,"<> ছাইছে লামি যাইতো, দাক্কা মারছে সামনে। আসসাল..."
9048,train_rangpur (497).wav,ফলান্তি। সবকিছু মনে করো এর মধ্যে <> এর খাওয়া এ...
11793,train_sylhet (2186).wav,যে বেশি এখবারে এখলা লাগতো৷ আস্তে আস্তে আলটিমেই...
8997,train_rangpur (446).wav,আমার নাম সাইদ। সাইদ? বয়স কত তোমার? এই ষোল। ষো...
453,train_sandwip (454).wav,"তোই, একারণে মুক্তি <> ইয়া মতি সাবের বৌ ক যাইবে..."
94,train_sandwip (95).wav,"গাইল দিলে তো মনে একটা জিদ চলি আইছে, মনে একটা জ..."
11594,train_sylhet (1987).wav,<> কিছু। আর আর অই যে নাবিদরে খইতাম আছলাম। কুনদ...
113,train_sandwip (114).wav,"রাজমিস্তুরির কাম কোইরবার হরে, তো আঁরে। টাইলসের..."
7092,train_narail (1153).wav,নাকি বিয়ে-শাদি অরবি? পড়া-লেহা কদ্দুর? পড়া-লেহা...


In [6]:
def extract_split(filename):
    filename_ = filename.split("_")
    split = filename_[0]
    return split

def extract_district(filename):
    filename_ = filename.split(" ")[0]
    district = filename_.split("_")[1]
    return district

def beautify_dataset(data):
    splits = []
    
    districts = []
    newpaths = []
    transcripts = []
    
    for i in range(len(data)):
        filename, transcript = data.iloc[i]
        split = extract_split(filename)
        district = extract_district(filename)
        dir_path = split2path[split]
        composed_path = f"{dir_path}{filename}"
        
        if os.path.exists(composed_path) == False:
            print(f"{composed_path} does not exist.")
            continue
        
        # replace any newline characters
        transcript = transcript.replace("\n", " ")
        transcript = " ".join(transcript.split())
        
        splits.append(split)
        districts.append(district)
        newpaths.append(composed_path)
        transcripts.append(transcript)
    
    data['file_path'] = newpaths
    data['district'] = districts
    data['split'] = splits
    data['transcripts'] = transcripts
    
#     data.drop(columns=['file_name'], inplace=True)
    
    return data

In [7]:
data = beautify_dataset(data)
data.sample(20)

,file_name,transcripts,file_path,district,split
10297,train_sylhet (690).wav,এর ফরে আরোখটা ওইলো গিয়া কুরবানির মাংস বিকালে ড...,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,sylhet,train
4073,train_habiganj (750).wav,"উনিশ, আটারো, সতেরো, সতেরো ওইলে ষোলর লাস্টে দেল...",/media/ahanaf/media-1/Datasets/ben10/ben10//16...,habiganj,train
4894,train_kishoreganj (624).wav,তে কত ইয়াদু দিতাছে। ইবার তো সব সংবাদ এইতা হাছা...,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,kishoreganj,train
3133,train_chittagong (1229).wav,"বেয়াকগুনে, খত আত্মীয়স্বজন আছিল। বেয়াকগুনে তো খ...",/media/ahanaf/media-1/Datasets/ben10/ben10//16...,chittagong,train
6355,train_narail (416).wav,তুই মানিস? সেকেন্ডবার তার ভোট চাওয়া লাগবে না। ...,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,narail,train
122,train_sandwip (123).wav,আইচ্ছা তো কাম অ্যাক্কান আমনেরে কন্ট্যাক্ট দিতো...,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,sandwip,train
8416,train_narsingdi (973).wav,চা খাওন লাগতো না। আইচ্ছা উডো সবাই উডো। সবাই উড...,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,narsingdi,train
2654,train_chittagong (750).wav,"আঁর আম্মুর আন্টি আছে দে ইবা আইসসি, আঁরা ছুডো আ...",/media/ahanaf/media-1/Datasets/ben10/ben10//16...,chittagong,train
8231,train_narsingdi (788).wav,তর এই আইপিএলের মাইজে আরে এই দিল্লির মাইজে। দিল...,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,narsingdi,train
2932,train_chittagong (1028).wav,তারফরে মাথা হঁরি উইট্টিতে এগিন মাথার মগজ ফাঁনি...,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,chittagong,train


In [8]:
data[data["transcripts"] == "<>"]

,file_name,transcripts,file_path,district,split
721,train_sandwip (722).wav,<>,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,sandwip,train
1296,train_barishal (242).wav,<>,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,barishal,train
1298,train_barishal (244).wav,<>,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,barishal,train
1299,train_barishal (245).wav,<>,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,barishal,train
1301,train_barishal (247).wav,<>,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,barishal,train
...,...,...,...,...,...
13330,train_tangail (778).wav,<>,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,tangail,train
13336,train_tangail (784).wav,<>,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,tangail,train
13356,train_tangail (804).wav,<>,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,tangail,train
13361,train_tangail (809).wav,<>,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,tangail,train


In [9]:
data[data["transcripts"] == ""]

,file_name,transcripts,file_path,district,split
3590,train_habiganj (267).wav,,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,habiganj,train
13505,train_tangail (953).wav,,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,tangail,train


In [10]:
data[data["transcripts"] == ".."]

,file_name,transcripts,file_path,district,split
1329,train_barishal (275).wav,..,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,barishal,train
1331,train_barishal (277).wav,..,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,barishal,train
1338,train_barishal (284).wav,..,/media/ahanaf/media-1/Datasets/ben10/ben10//16...,barishal,train


**NOTE:** Think of how you want use the existing models/your finetuned model to replace these examples.... For now let's just handle them.

In [11]:
# print(list(data[data['transcripts'] == ''].index))
data.drop(data[data['transcripts'] == ''].index, inplace=True)
      
# print(list(data[data['transcripts'] == '<>'].index))
data.drop(data[data['transcripts'] == "<>"].index, inplace=True)
      
# print(list(data[data['transcripts'] == '..'].index))
data.drop(data[data['transcripts'] == ".."].index, inplace=True)

In [12]:
data["transcripts"] = data["transcripts"].str.strip()

In [13]:
TASK = "transcribe"
MODEL_NAME = "openai/whisper-small"

In [14]:
feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_NAME, language='bn', task=TASK)
processor = WhisperProcessor.from_pretrained(MODEL_NAME, language='bn', task=TASK)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
ids = tokenizer.encode("")
ids

[50258, 50302, 50359, 50363, 50257]

In [16]:
tokenizer.decode(ids)

'<|startoftranscript|><|bn|><|transcribe|><|notimestamps|><|endoftext|>'

In [17]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        
        torch.cuda.empty_cache()

        return batch

In [18]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [19]:
def prepare_dataset(example):
    audio_path = example["file_path"]
    
    # load the audio using librosa or torch audio (as you wish)
    audio, sr = librosa.load(audio_path, sr=16_000)
    
    example["input_features"] = feature_extractor(audio, sampling_rate=sr).input_features[0]
    
    example["labels"] = tokenizer(f"{example['transcripts']}", max_length=448, padding=True, truncation=True).input_ids
    
    return example


def filter_inputs(input_audio):
    """filter inputs with zero input length"""
    return 0 < len(input_audio)


def filter_labels(input_labels):
    """filter empty label sequences"""
    return 0 < len(input_labels)

In [20]:
train_df = data[data["split"] == "train"]

In [21]:
"""
    adjust test size accordingly.
"""
train_df, eval_df = train_test_split(train_df, test_size=0.02, shuffle=True)

In [22]:
len(train_df), len(eval_df)

(13212, 270)

In [23]:
ben_reg_voice_ds = IterableDatasetDict()

train_split = DS.from_pandas(train_df)
eval_split = DS.from_pandas(eval_df)

ds_splits = IterableDatasetDict({
    'train': train_split,
    'eval': eval_split
})

In [24]:
ds_splits = ds_splits.remove_columns(["split"])

In [25]:
print(ds_splits)

IterableDatasetDict({
    train: Dataset({
        features: ['file_name', 'transcripts', 'file_path', 'district', '__index_level_0__'],
        num_rows: 13212
    })
    eval: Dataset({
        features: ['file_name', 'transcripts', 'file_path', 'district', '__index_level_0__'],
        num_rows: 270
    })
})


In [26]:
np.object = object

In [27]:
ds_splits = ds_splits.map(prepare_dataset,# remove_columns=ds_splits.column_names["train"],
                          #num_proc=2 # open for multithreadding
)

Map:   0%|          | 0/13212 [00:00<?, ? examples/s]

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

In [28]:
# ds_splits = ds_splits.filter(filter_inputs, input_columns=["input_features"])
# ds_splits = ds_splits.filter(filter_labels, input_columns=["labels"])

In [29]:
len(ds_splits["train"]), len(ds_splits["eval"])

(13212, 270)

In [30]:
cer = CharErrorRate()
wer = WordErrorRate()

In [31]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer_res = wer(pred_str, label_str)
    cer_res = cer(pred_str, label_str)
    
    """
        uncomment the next 3 lines if you want to see how the examples look like during eval 
    """
    print("WER:",wer_res,"| CER:", cer_res) # to show up during running logs
    print("Pred:",pred_str[0])
    print("Label:",label_str[0])
    
    return {"wer": wer_res, "cer": cer_res}

In [56]:
MODEL_NAME = './whisper-small-reg-ben-test'

In [57]:
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME, device_map="auto")

In [33]:
model_id = "whisper-small-reg-ben-test"

In [34]:
training_args = Seq2SeqTrainingArguments(
    output_dir=model_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    fp16=True,
    learning_rate=3e-4,
    weight_decay=1e-2,
    # warmup_steps=500,
    num_train_epochs=10,
    evaluation_strategy="steps", # or "epochs"
    predict_with_generate=True,
#     generation_max_length=448,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    report_to="none",
    remove_unused_columns=False,
    dataloader_num_workers=4,
    dataloader_prefetch_factor=32,
)

In [35]:
# eeee

In [36]:
model.generation_config.language = "bn"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None
model.config.suppress_tokens = [] # added later

In [37]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=ds_splits["train"],
    eval_dataset=ds_splits["eval"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(2, 1.0)]
)

/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [38]:
trainer.train()

# to use the high-level pipeline, ensure both the processor outputs and model outputs exist in the same dir
trainer.save_model(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)

  0%|          | 0/8260 [00:00<?, ?it/s]

/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


{'loss': 1.3785, 'grad_norm': 11.469520568847656, 'learning_rate': 2.88e-05, 'epoch': 0.06}
{'loss': 0.6753, 'grad_norm': 6.941266059875488, 'learning_rate': 5.88e-05, 'epoch': 0.12}
{'loss': 0.5553, 'grad_norm': 5.633580684661865, 'learning_rate': 8.879999999999999e-05, 'epoch': 0.18}
{'loss': 0.5149, 'grad_norm': 4.341473579406738, 'learning_rate': 0.0001188, 'epoch': 0.24}
{'loss': 0.5073, 'grad_norm': 6.691446304321289, 'learning_rate': 0.00014879999999999998, 'epoch': 0.3}
{'loss': 0.4922, 'grad_norm': 7.590158462524414, 'learning_rate': 0.00017879999999999998, 'epoch': 0.36}
{'loss': 0.4825, 'grad_norm': 4.008153438568115, 'learning_rate': 0.00020879999999999998, 'epoch': 0.42}
{'loss': 0.4945, 'grad_norm': 3.2979111671447754, 'learning_rate': 0.0002388, 'epoch': 0.48}
{'loss': 0.4952, 'grad_norm': 5.612159729003906, 'learning_rate': 0.0002688, 'epoch': 0.54}
{'loss': 0.4975, 'grad_norm': 5.593199253082275, 'learning_rate': 0.0002988, 'epoch': 0.61}
{'loss': 0.4781, 'grad_norm': 

  0%|          | 0/68 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


...

/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0034, 'grad_norm': 0.1698063462972641, 'learning_rate': 8.195876288659794e-06, 'epoch': 9.75}
{'loss': 0.003, 'grad_norm': 0.37620311975479126, 'learning_rate': 6.262886597938143e-06, 'epoch': 9.81}
{'loss': 0.0032, 'grad_norm': 0.24798984825611115, 'learning_rate': 4.3298969072164946e-06, 'epoch': 9.87}
{'loss': 0.0031, 'grad_norm': 0.2017114758491516, 'learning_rate': 2.396907216494845e-06, 'epoch': 9.93}
{'loss': 0.0029, 'grad_norm': 0.14242912828922272, 'learning_rate': 4.639175257731958e-07, 'epoch': 9.99}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


{'train_runtime': 19059.0599, 'train_samples_per_second': 6.932, 'train_steps_per_second': 0.433, 'train_loss': 0.19729342210144887, 'epoch': 10.0}


[]

In [39]:
out_logs = pd.DataFrame(trainer.state.log_history)
out_logs.to_csv("logs.csv")

In [ ]:
import gc

gc.collect()
del ds_splits

torch.cuda.empty_cache()

76

In [42]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model_id,
    chunk_length_s=30,
    device=0,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [43]:
def pretty_sort(filename):
    name, number_str = filename.split(" (")
    number = int(number_str.split(")")[0])
    return name, number

In [51]:
ids = []

In [52]:
from tqdm import tqdm

for root, dirs, files in os.walk("/media/ahanaf/media-1/Datasets/ben10/ben10/16_kHz_valid_audio"):
    files = sorted(files, key=pretty_sort)
    
#     print(files.index("valid_sandwip (1).wav"))
#     print(files.index("valid_sandwip (132).wav"))
    
#     put swandip first
    shift = files[1070 : 1202]
    
    files = shift + files[:1070] + files[1202:]
    ids = files.copy()
    preds = []
    print(f'Total: {len(files)}')
    for i, file in tqdm(enumerate(files), total=len(files), ncols=70, colour='green'):
        composed_path = f"{test_data_dir}{file}"
        audio, sr = librosa.load(composed_path, sr=16_000)
        text = pipe(audio)["text"]
        preds.append(text)
        if i % 50 == 0:
            print(f'{i+1:4d}/{len(files):4d}', file,'\t', text)

Total: 1703


  0%|                              | 1/1703 [00:02<1:11:19,  2.51s/it]

   1/1703 valid_sandwip (1).wav 	 হরালেখা করি কি অইবো, আঁই তো ইতাত উইট্টি যান লাগে না, আঁই যা আন ফিছে ইতাত লাগে না। জীবনের আন্নের হরালেখা যদি টার্গেট যদি আন্নের মানি দই যদি লোউখ্যস থিন্ন কর তাইলে তো কেমনে হইরবা


  3%|▊                            | 51/1703 [02:37<1:33:14,  3.39s/it]

  51/1703 valid_sandwip (51).wav 	 আত্তে বেশি বিরাইত্তান লাগে হিয়ান বেডার লগে দুই বিশ্তিত ফোন চইল্লে এক সাপ্তাহ ঠিক কাই চলাই আইবো। হিয়ান আই বেডার লগে কথা কনাই ঠিক নাই ফোন কইল্লে কইবো, আই হারাদিন কথা কমু নে খালি। কইবো খাইতো বইলে অতো খাইতো বইছে বেডা ফোন কচ্চে এরি অ�


...

 97%|██████████████████████████▏| 1651/1703 [1:14:10<02:33,  2.95s/it]

1651/1703 valid_tangail (80).wav 	 হ্যা, তো দিনে মনে করো যে মিনিমাম সাতটুক আদুস না করে টাইম দিতে অইয়া লাগবো, পথম কিনছে আমার, চিটাই অইয়া লাগবো। হ, টাইম তো দেহাই লাগবো। হ্যা, তো এইডাই অইলো নাই। তো আসার মইদ্যে থুইয়া শুরু কর।


100%|██████████████████████████▉| 1701/1703 [1:16:23<00:04,  2.28s/it]

1701/1703 valid_tangail (130).wav 	 কয়ডা বাজে? মেলা টাইম আছে। তরে কয়ডাই আমো? সমস্যা নাই। তুই কি করবি এহন? আমি দেহি একটু রুডির আনবার মন চাই। তাইলে যাইয়া রান্না করমু।


100%|███████████████████████████| 1703/1703 [1:16:27<00:00,  2.69s/it]


In [ ]:
sub_df = pd.DataFrame()
sub_df["id"] = ids
sub_df["sentence"] = preds
sub_df.to_csv("submission.csv", index=False)
sub_df.head(20)